You are given an excerpt of the log data from the [Swisslos](https://www.swisslos.ch) Jass servers collected between October 2017 and April 2018. In the Swiss card game [Jass](https://en.wikipedia.org/wiki/Jass), at the beginning of each round just after the cards have been dealt, the current player announces her choice of the trump mode from the options: diamonds, hearts, spades, club, obe-abe, une-ufe and push (only available when forehand). Use this information from human players to train a machine learning classifier in trump mode prediction.

In [16]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Import & Prepare Data

In [17]:
# # Import only a fraction of data for efficient testing
# data = pd.read_csv('data/2018_10_18_trump.csv', header=None, nrows=1000)

# Import all data in the final run
data = pd.read_csv("data/2018_10_18_trump_Test.csv", header=None)

data.head()

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0,0,0,0,1,0,1,1,0,0,...,1,0,0,0,1,0,0,0,53248,6
1,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,1,0,0,4613,5
2,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,68780,6
3,0,0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,24555,5
4,0,1,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,1,8392,4


In [18]:
data.shape

(359825, 39)

In [24]:
df = data.groupby(37).size().reset_index(name='counts')
df

,37,counts
0,0,94455
1,1,19
2,199,34
3,331,1091
4,345,50
5,349,1
6,382,131
7,409,1
8,421,424
9,427,99


In [25]:
df = df.drop(df[(df["counts"]>500)].index)[37]
df

1           1
2         199
4         345
5         349
6         382
7         409
8         421
9         427
10        430
11        437
12        471
13        478
14        479
16        501
17        513
18        517
19        522
20        538
21        542
23        572
24        575
25        597
26        600
27        604
28        608
29        621
30        633
31        657
32        661
33        675
        ...  
3461    83208
3462    83211
3463    83215
3464    83217
3465    83264
3466    83269
3467    83281
3468    83292
3469    83342
3470    83366
3471    83402
3472    83421
3473    83458
3474    83462
3475    83493
3476    83617
3477    83621
3478    83647
3479    83730
3480    83731
3481    83910
3482    83983
3483    84082
3484    84095
3485    84359
3486    84453
3487    85114
3488    85653
3489    85654
3490    85907
Name: 37, Length: 3399, dtype: int64

In [7]:
data = data.drop(df)
data

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0,0,0,0,1,0,1,1,0,0,...,1,0,0,0,1,0,0,0,53248,6
2,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,68780,6
3,0,0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,24555,5
4,0,1,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,1,8392,4
5,0,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,53076,5
6,1,1,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,41630,6
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,78376,1
8,0,1,1,0,0,0,1,0,1,0,...,0,0,0,1,0,0,1,1,55544,5
9,0,1,0,1,0,1,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
10,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,1,66261,2


In [ ]:
# Display number of rows and columns
data.shape

## Rename Columns for better Interpretability

In [ ]:
cards = [
# Diamonds
'DA','DK','DQ','DJ','D10','D9','D8','D7','D6',
# Hearts
'HA','HK','HQ','HJ','H10','H9','H8','H7','H6',
# Spades
'SA','SK','SQ','SJ','S10','S9','S8','S7','S6',
# Clubs
'CA','CK','CQ','CJ','C10','C9','C8','C7','C6'
]

# Forehand (yes = 1, no = 0)
forehand = ['FH']

user  = ['user']
trump = ['trump']

data.columns = cards + forehand + user + trump
data.head()

## Remove User Information

Later on you may want to keep this information. For now, we remove it just to avoid mistakes.

In [ ]:
data.drop('user', axis='columns', inplace=True)
data.head()

## Adjust Datatypes

In [ ]:
data.trump = data.trump.astype('category')
data[cards + forehand] = data[cards + forehand].astype(bool)
#data.dtypes

## Rename Target Category Values for better Interpretability

In [ ]:
# Backward compatibility: Value 10 for PUSH was used in an older version by Swisslos

data.trump.cat.rename_categories({0: 'DIAMONDS', 1: 'HEARTS', 2: 'SPADES', 3:'CLUBS',
                                  4: 'OBE_ABE', 5: 'UNE_UFE', 6: 'PUSH', 10: 'PUSH'}, inplace=True)
data.head()

# Data Quality Assessment

### Exercise: Sanity Check

Each row in the data now corresponds to one hand of a player plus her trump mode selection. Verify that each hand contains exactly 9 cards.

In [ ]:
# todo
data_cards = data[cards]
data = data[data_cards.sum(axis=1) == 9]
data

### Exercise: Statistical Fingerprint

Swisslos claims to distribute cards randomly.
* Verify that all card features roughly have the same mean and standard deviation.
* Over a **large number of rounds**, we would expect similar percentages for diamonds, hearts, spades and clubs as selected Trump. Verify this claim.

Hint: you may want to use [np.allclose(...)](https://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html) in combination with an assert statement for automated checking.

#### A note on randomness - not part of exercises: 

The above test is obviously a very poor guarantee of randomness. However, we perform such tests from the viewpoint of data quality rather than fairness. If you want to challenge the random number generator of Swisslos, use the NIST or Diehard test suite for randomness. Read more about random number tests <a href='https://gerhardt.ch/random.php'>here</a>. We have not done this ourself and keep believing that the provider of Swiss lottery use a reasonable source of randomness even for online games :-)

In [ ]:
# todo
diamonds = np.where(data.trump == 'DIAMONDS', 1, 0)
hearts = np.where(data.trump == 'HEARTS', 1, 0)
spades = np.where(data.trump == 'SPADES', 1, 0)
clubs = np.where(data.trump == 'CLUBS', 1, 0)
obe_abe = np.where(data.trump == 'OBE_ABE', 1, 0)
une_ufe = np.where(data.trump == 'UNE_UFE', 1, 0)
push = np.where(data.trump == 'PUSH', 1, 0)
print(diamonds.mean())
print(hearts.mean())
print(spades.mean())
print(clubs.mean())
print(obe_abe.mean())
print(une_ufe.mean())
print(push.mean())
print(diamonds.std())
print(hearts.std())
print(spades.std())
print(clubs.std())
print(obe_abe.std())
print(une_ufe.std())
print(push.std())

In [ ]:
data.trump.value_counts()

In [ ]:
data = data.drop(data[((data.trump == 'PUSH') & (data.index % 3 != 0))].index)  
data.head(15)

In [ ]:
data.trump.value_counts()

In [ ]:
diamonds = np.where(data.trump == 'DIAMONDS', 1, 0)
hearts = np.where(data.trump == 'HEARTS', 1, 0)
spades = np.where(data.trump == 'SPADES', 1, 0)
clubs = np.where(data.trump == 'CLUBS', 1, 0)
obe_abe = np.where(data.trump == 'OBE_ABE', 1, 0)
une_ufe = np.where(data.trump == 'UNE_UFE', 1, 0)
push = np.where(data.trump == 'PUSH', 1, 0)
print(diamonds.mean())
print(hearts.mean())
print(spades.mean())
print(clubs.mean())
print(obe_abe.mean())
print(une_ufe.mean())
print(push.mean())
print(diamonds.std())
print(hearts.std())
print(spades.std())
print(clubs.std())
print(obe_abe.std())
print(une_ufe.std())
print(push.std())

Can you explain ...
* the difference between Une-Ufe and Obe-Abe ?
* the difference between {spades, clubs} and {hearts, diamonds}

The fact that the values of the response are not distributed evenly has implications on the train-test split in the next section.

# Modelling and Prediction

### Train-Test Split

Split the available data into a training and a test set and put the test set aside for the final model evaluation. Use 20% of the data for testing, and set the random_state to 42 for reproducability. Since we found out that the classes are not distributed evenly, you need to stratify. Also, be aware that train_test_split() shuffles the data by default, which is what we want here (but not always, e.g. not in case of time series).

Your result should be the four variables *X_train, X_test, y_train, y_test*.

In [ ]:
from sklearn.model_selection import train_test_split

feature_columns = cards + forehand
X_train, X_test, y_train, y_test = train_test_split(data[feature_columns], data.trump, test_size=0.2,
                                                    stratify=data.trump, random_state=42)

## Baseline with a Logistic Regression Classifier

We create a (linear) logistic regression classifier and evaluate it with cross validation.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

classifier = LogisticRegression()

# Note that we do not use the test set here !
result = cross_val_score(classifier, X_train, y_train, cv=5, n_jobs=-1)

print("Mean accuracy over 5 folds is {:.4}".format(np.mean(result)))

### Exercise: Stochastic Gradient Descent

Machine learning classifiers internally use an optimization algorithm. Stochastic gradient descent is a stochastic 
approximation that generally is more efficient but with a slightly lower convergence rate. Change the code in the previous cell to a logistic regression with stochastic gradient descent. Use [SGDClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) with parameters *loss='log'* and *penalty='None'*

In [ ]:
from sklearn.linear_model import SGDClassifier

# todo
classifier = SGDClassifier(loss='log', penalty='None')

# Note that we do not use the test set here !
result = cross_val_score(classifier, X_train, y_train, cv=5, n_jobs=-1)

print("Mean accuracy over 5 folds is {:.4}".format(np.mean(result)))

## Learning Curves

Learning curves are an important diagnostic instrument in machine learning. A cross-validation generator splits the whole dataset k times into training and test data. Subsets of the training set with increasing sizes will be used to train the estimator and a score for each training subset size on the validation set will be computed. Afterwards, the scores will be averaged over all k runs for each training subset size.

As a rule of thumb, a gap between the training and validation curves indicates that using more data would improve accuracy.

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, valid_scores = learning_curve(SGDClassifier(loss='log', penalty='None'), 
                                                         X_train, y_train, n_jobs=-1,
                                                         train_sizes=np.linspace(0.1, 1.0, 50), cv=10,
                                                         exploit_incremental_learning=True, verbose=10)

In [ ]:
plot_data = pd.DataFrame({'train':train_scores.mean(axis=1), 'validation':valid_scores.mean(axis=1)}, index=train_sizes)
f, ax = plt.subplots(figsize=(14, 6))
ax.set_xlabel('#samples')
ax.set_ylabel('Accuracy')
ax.set_title('SGD')
plot_data.plot(ax=ax)
plt.grid()

### Exercise: Create a Random Forest Classifier

Now create a random forest classifier and evaluate it with cross validation.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# todo
classifier = RandomForestClassifier()

# Note that we do not use the test set here !
result = cross_val_score(classifier, X_train, y_train, cv=5, n_jobs=-1)

print("Mean accuracy over 5 folds is {:.4}".format(np.mean(result)))

### Exercise: List and Plot Feature Importances

Tree classsifiers (such as decision trees and random forests) can conveniently list feature importances. They use feature values to split the training set, whereas more important features are used higher in the tree(s). Use the *feature_importances_* attribute of a trained RandomForestClassifier to obtain a sorted list of the most important features. Plot feature importances in a bar chart.

In [ ]:
# todo
classifier.fit(X_train,y_train)
classifier.feature_importances_

## Grid Search over multiple Classifiers and Hyperparameters

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Classifier names
names = [
    "Logistic Regression",
    "Random Forest"
]

# Classifiers
classifiers = [
    LogisticRegression(solver='lbfgs', multi_class='multinomial'), # set these to avoid a FutureWarning with scikit-lean < 0.22
    RandomForestClassifier(n_estimators=200)
]

# Hyperparameter grid to search per classifier
parameters = [
    {'C': [0.5, 1.0, 2.0]},
    {'max_depth': [5, 10]}
]

list(zip(names, parameters))

In [ ]:
from sklearn.model_selection import GridSearchCV

results = []

for name, classifier, params in zip(names, classifiers, parameters):
    print("Grid search for {}".format(name))
    gs = GridSearchCV(classifier, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
    gs.fit(X_train, y_train)
    print("Best accuracy score found: {:.3f}\n".format(gs.best_score_))
    results.append([name, gs.best_score_, gs.best_estimator_])

results

Comment: If *refit=True* (the default), the best estimator is made available at the *best_estimator_* attribute of gs and permits using *.predict()* directly on this GridSearchCV instance. Note, however, that the above code runs grid search separately for each classifier, and so gs now only contains the last estimator of the list, which may or may not be the best one. So don't use gs to predict blindly.

### Exercise: Extend Grid Search by adding other Classifiers and Hyperparameters

You may want to experiment with a [nearest neighbors classifier](http://scikit-learn.org/stable/modules/neighbors.html) or a [support vector machine](http://scikit-learn.org/stable/modules/svm.html) or a [neural network](http://scikit-learn.org/stable/modules/neural_networks_supervised.html) or ...

### Exercise: Add interactions to Logistic Regression

A linear classifier such as logistic regression cannot model dependencies between features (things like "if your hand has both heart jack and heart nine, then choose heart as trump mode"). But features like these called *interactions* can be added manually.

In [ ]:
# First, get the score without interactions
clf = LogisticRegression(C=0.8, solver='lbfgs', multi_class='multinomial')
result = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1)
print("Mean accuracy over 5 folds is {:.4}".format(np.mean(result)))

For each of the four colors, add one interaction term for the combination jack and nine.

In [ ]:
for color in 'DHSC':
    # Jack and nine combination
    new_col = '{}_J9'.format(color)
    data[new_col]  = data['{}J'.format(color)] & data['{}9'.format(color)]
    feature_columns.append(new_col)
    
    # Exercise: Add other features here such as the combination of Ace-King-Queen (Dreiblatt).
    
data.head()

We must recreate training and test sets with the new features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[feature_columns], data.trump, test_size=0.2,
                                                    stratify=data.trump, random_state=42)

And check  if the additional columns improve the score

In [ ]:
clf = LogisticRegression(C=0.8, solver='lbfgs', multi_class='multinomial')
result = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1)
print("Mean accuracy over 5 folds is {:.4}".format(np.mean(result)))

### Exercise: Better Accuracy with less Data ?

At the beginning we removed the user ID from our data set. Use this information to develop a heuristic that seggregates between *good* and *bad* players. Can you improve accuracy by using only the data from *good* players?

In [ ]:
# no solution available

### Exercise: The final Score

Use the best setting you could find, train the classifier on all training data and evaluate on the yet unseen test data.

In [ ]:
# Probably not the best :-)
clf = LogisticRegression(C=0.5, solver='lbfgs', multi_class='multinomial')

clf.fit(X_train, y_train)
clf.score(X_test, y_test)